# Parrot Prediction Courses

## Evaluate results
In this notebook you will see how measure the quality of the algorithm performance.

**You will learn how to:**
- use predefined evaluation metrics,
- write your own evaluation metrics,
- use early stopping feature,
- cross validate results

https://github.com/dmlc/xgboost/blob/master/demo/guide-python/cross_validation.py

### Prepare data
Begin with loading all required libraries

In [1]:
import numpy as np
import xgboost as xgb

from pprint import pprint

Load agaricus dataset from file

In [2]:
# load Agaricus data
dtrain = xgb.DMatrix('../data/agaricus.txt.train')
dtest = xgb.DMatrix('../data/agaricus.txt.test')

Specify training parameters - we are going to use 5 stump decision trees with average learning rate.

In [3]:
# specify general training parameters
params = {
    'objective':'binary:logistic',
    'max_depth':1,
    'silent':1,
    'eta':0.5
}

num_rounds = 5

Before training the model let's specify `watchlist` to observe it's performance on the both datasets.

In [4]:
watchlist  = [(dtest,'test'), (dtrain,'train')]

### Using predefined evaluation metrics

#### What is already available?
There are already [some](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md) predefined metrics availabe. You can use them as the input for the `eval_metric` parameter while training the model.

- `rmse` - [root mean square error](https://www.wikiwand.com/en/Root-mean-square_deviation),
- `mae` - [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error?oldformat=true),
- `logloss` - [negative log-likelihood](https://en.wikipedia.org/wiki/Likelihood_function?oldformat=true)
- `error` - binary classification error rate. It is calculated as `#(wrong cases)/#(all cases)`. Treat predicted values with probability $p > 0.5$ as positive,
- `merror` - multiclass classification error rate. It is calculated as `#(wrong cases)/#(all cases)`,
- `auc` - [area under curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic?oldformat=true),
- `ndcg` - [normalized discounted cumulative gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain?oldformat=true),
- `map` - [mean average precision](https://en.wikipedia.org/wiki/Information_retrieval?oldformat=true)

By default an `error` metric will be used.

In [5]:
bst = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-error:0.110490	train-error:0.113926
[1]	test-error:0.110490	train-error:0.113926
[2]	test-error:0.033520	train-error:0.030401
[3]	test-error:0.027312	train-error:0.021495
[4]	test-error:0.031037	train-error:0.025487


To change is simply specify the `eval_metric` argument to the `params` dictionary.

In [6]:
params['eval_metric'] = 'logloss'
bst = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-logloss:0.457886	train-logloss:0.460115
[1]	test-logloss:0.383911	train-logloss:0.378731
[2]	test-logloss:0.312678	train-logloss:0.308064
[3]	test-logloss:0.269119	train-logloss:0.261390
[4]	test-logloss:0.239746	train-logloss:0.232175


You can also use multiple evaluation metrics at one time

In [7]:
params['eval_metric'] = ['logloss', 'auc']
bst = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-logloss:0.457886	test-auc:0.892138	train-logloss:0.460115	train-auc:0.888997
[1]	test-logloss:0.383911	test-auc:0.938901	train-logloss:0.378731	train-auc:0.942881
[2]	test-logloss:0.312678	test-auc:0.976157	train-logloss:0.308064	train-auc:0.981415
[3]	test-logloss:0.269119	test-auc:0.979685	train-logloss:0.261390	train-auc:0.985158
[4]	test-logloss:0.239746	test-auc:0.978500	train-logloss:0.232175	train-auc:0.983744


### Creating custom evaluation metric

In order to create our own evaluation metric, the only thing needed to do is to create a method taking two arguments - predicted probabilities, and `DMatrix` holding training data.

In this example our classification metric will simply count the number of misclassified examples assuming that classes with $p> 0.5$ are positive. You can change this threshold if you want more certainty.

In this case the algorithm is getting better when the number of misclassified examples is getting lower. Remember to also set the argument `maximize=False`.

In [8]:
# custom evaluation metric
def misclassified(pred_probs, dtrain):
    labels = dtrain.get_label() # obtain true labels
    preds = pred_probs > 0.5 # obtain predicted values
    return 'misclassified', np.sum(labels != preds)

In [9]:
bst = xgb.train(params, dtrain, num_rounds, watchlist, feval=misclassified, maximize=False)

[0]	test-misclassified:178.000000	train-misclassified:742.000000
[1]	test-misclassified:178.000000	train-misclassified:742.000000
[2]	test-misclassified:54.000000	train-misclassified:198.000000
[3]	test-misclassified:44.000000	train-misclassified:140.000000
[4]	test-misclassified:50.000000	train-misclassified:166.000000


You can see that even though the `params` dictionary is holding `eval_metric` key these values are being ignored and overwritten by `feval`.

### Playing with evaluation results
You can get evaluation scores by declaring a dictionary for holding values and passing it as a parameter for `evals_result` argument

In [10]:
evals_result = {}
bst = xgb.train(params, dtrain, num_rounds, watchlist, feval=misclassified, maximize=False, evals_result=evals_result)

[0]	test-misclassified:178.000000	train-misclassified:742.000000
[1]	test-misclassified:178.000000	train-misclassified:742.000000
[2]	test-misclassified:54.000000	train-misclassified:198.000000
[3]	test-misclassified:44.000000	train-misclassified:140.000000
[4]	test-misclassified:50.000000	train-misclassified:166.000000


Now you can reuse these scores (ie. for plotting)

In [11]:
pprint(evals_result)

{'test': {'misclassified': ['178.000000',
                            '178.000000',
                            '54.000000',
                            '44.000000',
                            '50.000000']},
 'train': {'misclassified': ['742.00000',
                             '742.00000',
                             '198.00000',
                             '140.00000',
                             '166.00000']}}


### Early stopping
There is a nice optimization trick that you can use when fitting multiple trees. 

You can train the model until the validation score stops improving. Validation error needs to decrease at least every `early_stopping_rounds` to continue training. This approach results in simpler model, because the best number of trees will be found.

In the following example a total number of 1500 trees is to be creeated, but we are telling to stop if the validation score does not improve for last ten iterations.

In [15]:
params['eval_metric'] = 'error'
num_rounds = 1500

bst = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10)

Will train until train error hasn't decreased in 10 rounds.
[0]	test-error:0.110490	train-error:0.113926
[1]	test-error:0.110490	train-error:0.113926
[2]	test-error:0.033520	train-error:0.030401
[3]	test-error:0.027312	train-error:0.021495
[4]	test-error:0.031037	train-error:0.025487
[5]	test-error:0.019243	train-error:0.017350
[6]	test-error:0.019243	train-error:0.017350
[7]	test-error:0.015518	train-error:0.013358
[8]	test-error:0.015518	train-error:0.013358
[9]	test-error:0.009311	train-error:0.007523
[10]	test-error:0.015518	train-error:0.013358
[11]	test-error:0.019243	train-error:0.017350
[12]	test-error:0.009311	train-error:0.007523
[13]	test-error:0.001862	train-error:0.001996
[14]	test-error:0.005587	train-error:0.005988
[15]	test-error:0.005587	train-error:0.005988
[16]	test-error:0.005587	train-error:0.005988
[17]	test-error:0.005587	train-error:0.005988
[18]	test-error:0.005587	train-error:0.005988
[19]	test-error:0.005587	train-error:0.005988
[20]	test-error:0.005587	train

When using `early_stopping_rounds` parameter resulting model will have 3 additional fields - `bst.best_score`, `bst.best_iteration` and `bst.best_ntree_limit`.

In [19]:
print("Booster best train score: {}".format(bst.best_score))
print("Booster best iteration: {}".format(bst.best_iteration))
print("Booster best number of trees limit: {}".format(bst.best_ntree_limit))

Booster best train score: 0.001996
Booster best iteration: 13
Booster best number of trees limit: 14


Also keep in mind that `train()` will return a model from the last iteration, not the best one.

### Cross validating results